<a href="https://colab.research.google.com/github/erikaduda/dsto-gan/blob/main/Pr%C3%A9_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Programa em python para ler um arquivo separar os atributos da classe. Se os atributos forem nominais aplicar codificação one-hot. E se a classe para nominal substituir cada item da classe por números sequenciais (começando por 0). Depois ler o arquivo e verificar se há atributos vazios ou nulos. Se tiver substitua pela média. Gravar o resultado num arquivo .csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import re
import sklearn
import os
import tensorflow as tf
import glob
from google.colab import files
from pandas import DataFrame
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from scipy.sparse import csr_matrix

# Função para aplicar codificação one-hot em atributos nominais
def apply_one_hot_encoding(data, column):
    encoded_data = pd.get_dummies(data[column], prefix=column)
    data = pd.concat([data, encoded_data], axis=1)
    data.drop(column, axis=1, inplace=True)
    return data

# Função para substituir atributos vazios pela média
def replace_missing_values(data):
    data.replace("?", np.nan, inplace=True)  # Substituir "?" por NaN
    imputer = SimpleImputer(strategy="mean")
    data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    return data_imputed

PATH = ("/content/drive/MyDrive/PHD_new/dataset/falta/")

csv_files = glob.glob(os.path.join(PATH, "*.csv"))
print(csv_files)
print(type(csv_files))

for path, dirs, files in os.walk(PATH):
    dirs = sorted(dirs)
    files = sorted(files)
    break

for i in range(len(files)):
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>")
    print("COUNT FILE: ", i)
    print("NAME FILE:", files[i])

    # Leitura do arquivo
    try:
        data = pd.read_csv(PATH + files[i])
    except pd.errors.ParserError as e:
        print(f"Error parsing file {files[i]}: {str(e)}")
        continue

    # Separar os atributos da classe
    class_column = 'class'
    class_data = data[class_column]
    feature_data = data.drop(class_column, axis=1)

    # Verificar se os atributos são nominais e aplicar codificação one-hot se necessário
    for column in feature_data.columns:
        if feature_data[column].dtype == object:
            feature_data = apply_one_hot_encoding(feature_data, column)

    # Substituir valores "?" por média dos valores
    feature_data = replace_missing_values(feature_data)

    # Substituir classe nominal por números sequenciais ou valores binários (True=1, False=0)
    if class_data.dtype == bool:
        class_data = class_data.astype(int)  # Converte bool para int (0 e 1)
    elif class_data.dtype == object:
        class_data = class_data.map({'True': 1, 'False': 0})  # Mapeia 'True' para 1 e 'False' para 0

    # Substituir numerica por números sequenciais
    if class_data.dtype == int:
        sorted_values = sorted(class_data)
        if all(sorted_values[i] == sorted_values[0] + i for i in range(len(sorted_values))):  # Verifica se os valores são sequenciais
            class_data = class_data - sorted_values[0]  # Subtrai o primeiro valor para começar de 0
        else:
            label_encoder = LabelEncoder()
            class_data = label_encoder.fit_transform(class_data)


   # Substituir classe nominal por números sequenciais ou valores binários (True=1, False=0)
    if class_data.dtype == bool:
        class_data = class_data.astype(int)  # Converte bool para int (0 e 1)
    elif class_data.dtype == object:
        class_data = class_data.map({'True': 1, 'False': 0})  # Mapeia 'True' para 1 e 'False' para 0

    # Reunir atributos e classe novamente
    data_encoded = pd.concat([feature_data, pd.Series(class_data, name=class_column)], axis=1)

    # Gravar o resultado em um arquivo CSV
    data_encoded.to_csv('/content/drive/MyDrive/PHD_new/dataset_tratado/error/' + files[i], index=False)


['/content/drive/MyDrive/PHD_new/dataset/falta/bank-additional-full.csv']
<class 'list'>
>>>>>>>>>>>>>>>>>>>>>>>>>>
COUNT FILE:  0
NAME FILE: bank-additional-full.csv
